# MongoDB Design Patterns (in Java)

Install dependencies

In [55]:
%maven org.mongodb:mongodb-driver-sync:5.2.1
%maven org.json:json:20240303

In [20]:
import org.bson.Document;

import com.mongodb.client.MongoClient;
import com.mongodb.client.MongoClients;
import com.mongodb.client.MongoCollection;
import com.mongodb.client.MongoDatabase;
import java.util.List;
import com.mongodb.ExplainVerbosity;

String uri = "mongo-uri-here"; // REPLACE ME


In [88]:
// POJO Classes
class TruckDriver {
    public int _id;
    public String name;
    public String license;
    public int experience;
}

class Capacity {
    public int value;
    public String unit;
}

class Length {
    public int value;
    public String unit;
}

class Width {
    public int value;
    public String unit;
}

class Height {
    public int value;
    public String unit;
}

class Dimensions {
    public Length length;
    public Width width;
    public Height height;
}

class AttributesV1 {
    public String color;
    public String model;
    public int year;
    public int mileage;
    public String[] dimensions;
}

class AttributesV2 {
    public String color;
    public String model;
    public int year;
    public int mileage;
    public Dimensions dimensions;
}

class EquipmentV1 {
    public int _id;
    public String type;
    public String capacity;
    public String fuelType;
    public TruckDriver driver;
    public int axles;
    public String route;
    public String status;
    public AttributesV1 attributes;
    public int version;
}

class EquipmentV2 {
    public int _id;
    public String type;
    public Capacity capacity;
    public String fuelType;
    public TruckDriver driver;
    public int axles;
    public String route;
    public String status;
    public AttributesV2 attributes;
    public int version;
}

class Route {
    public String route;
    public List<Integer> equipments;
    public int distance;
    public List<String> stops;
}

class IoTData {
    public int truckId;
    public java.util.Date timestamp;
    public double temperature;
    public double humidity;
}

Initialize the connection

In [23]:
MongoClient mongoClient = MongoClients.create(uri);
MongoDatabase database = mongoClient.getDatabase("design-patterns");

Debug function to pretty-print JSON

In [61]:
import com.google.gson.Gson;
import com.google.gson.GsonBuilder;

public class JsonUtils {
    public static String prettyPrintJson(String json) {
        Gson gson = new GsonBuilder().setPrettyPrinting().create();
        return gson.toJson(gson.fromJson(json, Object.class));
    }
}

The full driver object contains lots of data

In [ ]:
// Subset Pattern
MongoCollection<Document> driversCollection = database.getCollection("drivers");
// get one driver with aggregation pipeline and show explain plan
Document driverDoc = driversCollection.aggregate(List.of(
    new Document("$limit", 1)
)).first();

if (driverDoc != null) {
    System.out.println("=== Subset Pattern - Full Driver ===");
    System.out.println(JsonUtils.prettyPrintJson(driverDoc.toJson()));
}

In the equipements collection, we only keep minimal driver information

In [66]:
MongoCollection<Document> equipmentsCollection = database.getCollection("equipments");
Document equipmentDoc = equipmentsCollection.aggregate(List.of(
    new Document("$limit", 1)
)).first();

if (equipmentDoc != null) {
    System.out.println("=== Subset Pattern - Full Equipment ===");
    System.out.println(JsonUtils.prettyPrintJson(equipmentDoc.toJson()));
}

// now show the explain plan and pretty-print it
explainPlan = equipmentsCollection.aggregate(List.of(
    new Document("$limit", 1)
)).explain(ExplainVerbosity.EXECUTION_STATS);

if (explainPlan != null) {
    System.out.println("=== Explain Plan ===");
    System.out.println(JsonUtils.prettyPrintJson(explainPlan.toJson()));
}


=== Subset Pattern - Full Equipment ===
{
  "_id": 1.0,
  "type": "Truck",
  "capacity": {
    "value": 15.0,
    "unit": "tons"
  },
  "fuelType": "Diesel",
  "driver": {
    "driverId": 1.0,
    "name": "John Doe"
  },
  "axles": 2.0,
  "route": "Route 66",
  "status": "active",
  "attributes": {
    "color": "red",
    "model": "Volvo",
    "year": 2018.0,
    "mileage": 120000.0,
    "dimensions": {
      "length": {
        "value": 10.0,
        "unit": "m"
      },
      "width": {
        "value": 2.5,
        "unit": "m"
      },
      "height": {
        "value": 3.0,
        "unit": "m"
      }
    }
  },
  "version": 2.0
}
=== Explain Plan ===
{
  "explainVersion": "1",
  "queryPlanner": {
    "namespace": "design-patterns.equipments",
    "parsedQuery": {},
    "indexFilterSet": false,
    "planCacheShapeHash": "8F2383EE",
    "planCacheKey": "7DF350EE",
    "optimizationTimeMillis": 0.0,
    "optimizedPipeline": true,
    "maxIndexedOrSolutionsReached": false,
    "maxInd

And we avoid doing a lookup to get the details

In [ ]:
equipmentDoc = equipmentsCollection.aggregate(List.of(
    new Document("$limit", 1),
    new Document("$lookup", new Document("from", "drivers").append("localField", "driver.driverId").append("foreignField", "_id").append("as", "driver"))
)).first();

if (equipmentDoc != null) {
    System.out.println("=== Subset Pattern - Full Equipment with Driver ===");
    System.out.println(JsonUtils.prettyPrintJson(equipmentDoc.toJson()));
}

// now show the explain plan and pretty-print it
explainPlan = equipmentsCollection.aggregate(List.of(
    new Document("$limit", 1),
    new Document("$lookup", new Document("from", "drivers").append("localField", "driverId").append("foreignField", "_id").append("as", "driver"))
)).explain(ExplainVerbosity.EXECUTION_STATS);

if (explainPlan != null) {
    System.out.println("=== Explain Plan ===");
    System.out.println(JsonUtils.prettyPrintJson(explainPlan.toJson()));
}

=== Subset Pattern - Full Equipment with Driver ===
{
  "_id": 1.0,
  "type": "Truck",
  "capacity": {
    "value": 15.0,
    "unit": "tons"
  },
  "fuelType": "Diesel",
  "driver": [
    {
      "_id": 1.0,
      "name": "John Doe",
      "license": "A1234567",
      "experience": 5.0
    }
  ],
  "axles": 2.0,
  "route": "Route 66",
  "status": "active",
  "attributes": {
    "color": "red",
    "model": "Volvo",
    "year": 2018.0,
    "mileage": 120000.0,
    "dimensions": {
      "length": {
        "value": 10.0,
        "unit": "m"
      },
      "width": {
        "value": 2.5,
        "unit": "m"
      },
      "height": {
        "value": 3.0,
        "unit": "m"
      }
    }
  },
  "version": 2.0
}


=== Explain Plan ===
{
  "explainVersion": "2",
  "queryPlanner": {
    "namespace": "design-patterns.equipments",
    "parsedQuery": {},
    "indexFilterSet": false,
    "planCacheShapeHash": "B3956D95",
    "planCacheKey": "E66D7B71",
    "optimizationTimeMillis": 0.0,
    "optimizedPipeline": true,
    "maxIndexedOrSolutionsReached": false,
    "maxIndexedAndSolutionsReached": false,
    "maxScansToExplodeReached": false,
    "prunedSimilarIndexes": false,
    "winningPlan": {
      "isCached": false,
      "queryPlan": {
        "stage": "EQ_LOOKUP",
        "planNodeId": 3.0,
        "foreignCollection": "design-patterns.drivers",
        "localField": "driverId",
        "foreignField": "_id",
        "asField": "driver",
        "strategy": "IndexedLoopJoin",
        "indexName": "_id_",
        "indexKeyPattern": {
          "_id": 1.0
        },
        "scanDirection": "forward",
        "inputStage": {
          "stage": "LIMIT",
          "planNodeId": 2.0,
          "limit

In [76]:
// Base class with common fields
class Vehicle {
    public int _id;
    public String type;
    public Capacity capacity;
    public String fuelType;
    public String route;
    public String status;
    public Attributes attributes;
    public int version;
}

// POJO class for a Truck with the proper subset of fields
class Truck extends Vehicle {
    public TruckDriver driver;
    public int axles;
}

// POJO class for a Train with the proper subset of fields
class Train extends Vehicle {
    public int cars;
}

With this inheritance defined, we can now fetch specific types

In [83]:
// Polymorphic Pattern

var truck = equipmentsCollection.find(new Document("type", "Truck")).first();
if (truck != null) {
  System.out.println("=== Polymorphic Pattern - Truck ===");
  System.out.println(truck);
}

=== Polymorphic Pattern - Truck ===
Document{{_id=1, type=Truck, capacity=Document{{value=15, unit=tons}}, fuelType=Diesel, driver=Document{{driverId=1, name=John Doe}}, axles=2, route=Route 66, status=active, attributes=Document{{color=red, model=Volvo, year=2018, mileage=120000, dimensions=Document{{length=Document{{value=10, unit=m}}, width=Document{{value=2.5, unit=m}}, height=Document{{value=3, unit=m}}}}}}, version=2}}


In [93]:
// Schema Versioning Pattern
Document versionedDoc = equipmentsCollection.find(new Document("version", 2)).first();
if (versionedDoc != null) {
    EquipmentV2 equipmentV2 = new EquipmentV2();
    equipmentV2._id = versionedDoc.getInteger("_id");
    equipmentV2.type = versionedDoc.getString("type");
    //equipmentV2.capacity = versionedDoc.get("capacity", Capacity.class);
    equipmentV2.fuelType = versionedDoc.getString("fuelType");
    //equipmentV2.driver = versionedDoc.get("driver", TruckDriver.class);
    equipmentV2.axles = versionedDoc.getInteger("axles");
    equipmentV2.route = versionedDoc.getString("route");
    equipmentV2.status = versionedDoc.getString("status");
    //equipmentV2.attributes = versionedDoc.get("attributes", AttributesV2.class);
    equipmentV2.version = versionedDoc.getInteger("version");
    System.out.println("Schema Versioning Pattern - Version 2 Equipment: " + equipmentV2);
}

Schema Versioning Pattern - Version 2 Equipment: REPL.$JShell$50L$EquipmentV2@5884167e


In [94]:


// Bucket Pattern
MongoCollection<Document> routesCollection = database.getCollection("routes");
Document routeDoc = routesCollection.find().first();
if (routeDoc != null) {
    System.out.println("Bucket Pattern - Route: " + routeDoc.toJson());
}

// Time Series Pattern
MongoCollection<Document> iotDataCollection = database.getCollection("iotData");
Document iotDataDoc = iotDataCollection.find().first();
if (iotDataDoc != null) {
    System.out.println("Time Series Pattern - IoT Data: " + iotDataDoc.toJson());
}

Bucket Pattern - Route: {"_id": {"$oid": "6763d5c930415c56a91acc1d"}, "route": "Route 66", "equipments": [1, 4, 6, 9], "distance": 2450, "stops": ["Chicago", "St. Louis", "Oklahoma City"]}


Time Series Pattern - IoT Data: {"timestamp": {"$date": "2024-12-17T23:55:01.998Z"}, "truckId": 194, "humidity": 81.95186938811298, "_id": {"$oid": "6763d5ca30415c56a91ad3b2"}, "temperature": 8.11419715222735}
